In [1]:
from __future__ import print_function
import ipywidgets as widgets
from transformers import pipeline
# from simpletransformers.t5 import T5Model, T5Args
import pandas as pd
import logging
import numpy as np
import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
import json
import random
# Pull reddit data from reddit api
import requests
from bs4 import BeautifulSoup
import math
#tf for RNN
import tensorflow_datasets as tfds
import tensorflow as tf

In [2]:
# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('pigKA_TKnDkXcatEGcbo8g', 'nawGKK2MfPtC6vKz8TjaNEnmYfAggA')

# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': 'Katsuuu100',
        'password': 'Testing159753'}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyBot/0.0.1'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [3]:
# HTML web scraper, scrape top subreddits, SFW only.
# 
# Source: https://realpython.com/beautiful-soup-web-scraper-python/
# Source: https://stackoverflow.com/questions/40210093/how-do-i-scrape-only-div-class-quotetext-from-a-website-using-python


URL = "http://redditlist.com/sfw/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
# print(soup.prettify())
job_elements = soup.find_all("div", class_="listing-item")

# print(job_elements[0])

# Filter to only the Top 125 Most subscribed subreddits
job_elements = job_elements[125:250]

master_subreddit_list = []
# Find subreddit names
for job_element in job_elements:
  links = job_element.find_all("a")
  for link in links:
    print(link.text.strip())
    master_subreddit_list.append(link.text.strip())

announcements
funny
AskReddit
gaming
aww
Music
pics
worldnews
movies
science
todayilearned
videos
news
Showerthoughts
Jokes
food
askscience
IAmA
EarthPorn
gifs
nottheonion
books
DIY
Art
explainlikeimfive
LifeProTips
space
sports
mildlyinteresting
Documentaries
gadgets
memes
tifu
photoshopbattles
UpliftingNews
GetMotivated
dataisbeautiful
listentothis
history
philosophy
television
InternetIsBeautiful
Futurology
WritingPrompts
OldSchoolCool
personalfinance
nosleep
creepy
TwoXChromosomes
wallstreetbets
technology
wholesomememes
AdviceAnimals
interestingasfuck
Fitness
politics
lifehacks
WTF
oddlysatisfying
relationship_advice
NatureIsFuckingLit
Minecraft
travel
facepalm
Whatcouldgowrong
nextfuckinglevel
pcmasterrace
leagueoflegends
Unexpected
me_irl
BlackPeopleTwitter
buildapc
dadjokes
dankmemes
bestof
PS4
Tinder
place
MadeMeSmile
AnimalsBeingBros
Damnthatsinteresting
tattoos
CryptoCurrency
AnimalsBeingJerks
nba
photography
AnimalsBeingDerps
gardening
BikiniBottomTwitter
trippinthroughtime

In [4]:
# Pick 5 random subreddits
five_random_subreddits = random.choices(master_subreddit_list, k=5)
print(five_random_subreddits)

['tifu', 'programming', 'EarthPorn', 'news', 'Damnthatsinteresting']


In [5]:
# Source: https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c
# Source: https://pynative.com/python-random-choice/
# Pull from 5 classes. 5 classes = 5 subreddits.

my_list_of_dictionaries = []
total = 0
# Target Count for test set = 100.
test_set_count = 20
n = int(math.ceil(test_set_count/0.20/25))

# url_list_check = [f"https://oauth.reddit.com/r/{five_random_subreddits[0]}/new/",
#           f"https://oauth.reddit.com/r/{five_random_subreddits[1]}/new/",
#           f"https://oauth.reddit.com/r/{five_random_subreddits[2]}/new/",
#           f"https://oauth.reddit.com/r/{five_random_subreddits[3]}/new/",
#           f"https://oauth.reddit.com/r/{five_random_subreddits[4]}/new/"
#           ]

url_list_check = [f"https://oauth.reddit.com/r/wallstreetbets/new/",
          f"https://oauth.reddit.com/r/teenagers/new/",
          f"https://oauth.reddit.com/r/copypasta/new/",
          f"https://oauth.reddit.com/r/genz/new/",
          f"https://oauth.reddit.com/r/unpopularopinion/new/",
          # f"https://oauth.reddit.com/r/frat/new/"
          ]

for i in range(len(url_list_check)):
    
  # print(url_list_check[i])
  res_check = requests.get(url_list_check[i],
                    headers=headers,
                    params={"limit": "1"})
  
  # print(res_check)
  # print(json.dumps(res_check.json()["data"]["children"][0]["data"]["name"], indent=4))
  name = res_check.json()["data"]["children"][0]["data"]["name"]
  page_count = 25
  
  for j in range(n):

    # url_list = [f"https://oauth.reddit.com/r/{five_random_subreddits[0]}/new/?count={page_count}&after={name}",
    #         f"https://oauth.reddit.com/r/{five_random_subreddits[1]}/new/?count={page_count}&after={name}",
    #         f"https://oauth.reddit.com/r/{five_random_subreddits[2]}/new/?count={page_count}&after={name}",
    #         f"https://oauth.reddit.com/r/{five_random_subreddits[3]}/new/?count={page_count}&after={name}",
    #         f"https://oauth.reddit.com/r/{five_random_subreddits[4]}/new/?count={page_count}&after={name}"
    #         ]

    url_list = [f"https://oauth.reddit.com/r/wallstreetbets/new/?count={page_count}&after={name}",
            f"https://oauth.reddit.com/r/teenagers/new/?count={page_count}&after={name}",
            f"https://oauth.reddit.com/r/copypasta/new/?count={page_count}&after={name}",
            f"https://oauth.reddit.com/r/genz/new/?count={page_count}&after={name}",
            f"https://oauth.reddit.com/r/unpopularopinion/new/?count={page_count}&after={name}",
            # f"https://oauth.reddit.com/r/frat/new/?count={page_count}&after={name}"
            ]
    
    print("Page Count:", page_count)
    print("Name:", name)
    print("Url:", url_list[i])
    
    res = requests.get(url_list[i],
                    headers=headers)
                    # params={"limit": "100"})

    reddit_dictionary = res.json()

    for k in range(len(reddit_dictionary["data"]["children"])):
      my_dictionary = {}
      my_dictionary["subreddit"] = reddit_dictionary["data"]["children"][k]["data"]["subreddit"]
      my_dictionary["text"] = reddit_dictionary["data"]["children"][k]["data"]["selftext"]
      # If a reddit post is has no body text:
      if my_dictionary["text"] == "":
        # Replace with title of reddit post.
        my_dictionary["text"] = reddit_dictionary["data"]["children"][k]["data"]["title"]
      
      print(my_dictionary["text"])
      # my_dictionary["title"] = reddit_dictionary["data"]["children"][k]["data"]["title"]
      # my_dictionary["url"] = reddit_dictionary["data"]["children"][k]["data"]["url"]
      # print(reddit_dictionary["data"]["children"][k]["data"]["subreddit"])
      # print(reddit_dictionary["data"]["children"][k]["data"]["selftext"])
      # print(reddit_dictionary["data"]["children"][k]["data"]["url"])
      my_list_of_dictionaries.append(my_dictionary)
      total += 1
      name = reddit_dictionary["data"]["children"][k]["data"]["name"]
    # print(json.dumps(my_list_of_dictionaries, indent=4, sort_keys=False))
    
    page_count += 25

print("Total gathered:", total)

Page Count: 25
Name: t3_tyny3r
Url: https://oauth.reddit.com/r/wallstreetbets/new/?count=25&after=t3_tyny3r
Fidelity thinks 36 trades a year should require approval. What should I tell them?
congrats on new car, show yourself retard
Fuck Your Puts
Read more: [https://updateonbitcoin.com/5-cryptocurrencies-that-should-rise-more-than-500-by-march/](https://updateonbitcoin.com/5-cryptocurrencies-that-should-rise-more-than-500-by-march/)

&amp;#x200B;

https://preview.redd.it/w57arjozg6s81.png?width=1280&amp;format=png&amp;auto=webp&amp;s=7376c5cbb1bcc2395ee1d018e3ee0730e88c497f
[Here's](https://www.reddit.com/r/wallstreetbets/comments/tykmcq/use_margin_loans_to_pay_rent/) the thread for context.

Look at the top responses to anon asking about using 10K of margin for rent:

[https://imgur.com/a/0IUEd4b](https://imgur.com/a/0IUEd4b)

I'm sorry, when the FUCK did you all become such pussies? Margin can be an extremely useful and affordable tool to manage your cash flows and increase returns.

In [6]:
# Source: https://github.com/susanli2016/NLP-with-Python/blob/master/Text_Classification_With_BERT.ipynb
# Convert list of dictionaries into pandas df
df = pd.DataFrame(my_list_of_dictionaries)
df.head()

,subreddit,text
0,wallstreetbets,Fidelity thinks 36 trades a year should requir...
1,wallstreetbets,"congrats on new car, show yourself retard"
2,wallstreetbets,Fuck Your Puts
3,wallstreetbets,Read more: [https://updateonbitcoin.com/5-cryp...
4,wallstreetbets,[Here's](https://www.reddit.com/r/wallstreetbe...


In [7]:
# Check if there is body text in reddit post
print(df['text'][0])

Fidelity thinks 36 trades a year should require approval. What should I tell them?


In [8]:
# Check if total is correct per subreddit
df['subreddit'].value_counts()

wallstreetbets      100
teenagers           100
copypasta           100
GenZ                100
unpopularopinion    100
Name: subreddit, dtype: int64

In [9]:
possible_labels = df.subreddit.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'wallstreetbets': 0,
 'teenagers': 1,
 'copypasta': 2,
 'GenZ': 3,
 'unpopularopinion': 4}

In [10]:
df['label'] = df.subreddit.replace(label_dict)

In [11]:
df.head()

,subreddit,text,label
0,wallstreetbets,Fidelity thinks 36 trades a year should requir...,0
1,wallstreetbets,"congrats on new car, show yourself retard",0
2,wallstreetbets,Fuck Your Puts,0
3,wallstreetbets,Read more: [https://updateonbitcoin.com/5-cryp...,0
4,wallstreetbets,[Here's](https://www.reddit.com/r/wallstreetbe...,0


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.20, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

In [13]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [14]:
df.groupby(['subreddit', 'label', 'data_type']).count()

text
subreddit        label data_type      
GenZ             3     train        80
                       val          20
copypasta        2     train        80
                       val          20
teenagers        1     train        80
                       val          20
unpopularopinion 4     train        80
                       val          20
wallstreetbets   0     train        80
                       val          20

In [15]:
df.head()

,subreddit,text,label,data_type
0,wallstreetbets,Fidelity thinks 36 trades a year should requir...,0,train
1,wallstreetbets,"congrats on new car, show yourself retard",0,val
2,wallstreetbets,Fuck Your Puts,0,val
3,wallstreetbets,Read more: [https://updateonbitcoin.com/5-cryp...,0,train
4,wallstreetbets,[Here's](https://www.reddit.com/r/wallstreetbe...,0,train


In [39]:
train_dataset = df[df.data_type == 'train']
val_dataset = df[df.data_type == 'val']

train_dataset = train_dataset.drop(['subreddit','data_type'], axis = 1)
val_dataset = val_dataset.drop(['subreddit','data_type'], axis = 1)
train_X, train_Y = train_dataset['text'], train_dataset['label']
test_X, test_Y = val_dataset['text'], val_dataset['label']
hot_train_Y = pd.get_dummies(train_dataset.label)
hot_test_Y = pd.get_dummies(val_dataset.label)
hot_test_Y


,0,1,2,3,4
1,1,0,0,0,0
2,1,0,0,0,0
14,1,0,0,0,0
20,1,0,0,0,0
21,1,0,0,0,0
...,...,...,...,...,...
462,0,0,0,0,1
467,0,0,0,0,1
485,0,0,0,0,1
491,0,0,0,0,1


In [105]:
#make RNN model

model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=128,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    #bidirectional expects 3D input
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=False)),
    tf.keras.layers.Dense(32,activation = 'relu'),
    tf.keras.layers.Dense(5,activation = 'softmax'), 
    # tf.keras.layers.Dense(1, activation = 'softmax')
])

In [110]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(1e-3),
              metrics=['accuracy'])
        
        

In [112]:
#training not working because the input doesn't fit. There shouldn't be anything empty, null, or Nan so maybe an exploding gradient?
history = model.fit(train_X, hot_train_Y, epochs=3, batch_size = 64, validation_data=(test_X, hot_test_Y))

Epoch 1/3
7/7 [==============================] - 28s 3s/step - loss: 1.3752 - accuracy: 0.5375 - val_loss: 1.4637 - val_accuracy: 0.4100
Epoch 2/3
7/7 [==============================] - 15s 2s/step - loss: 1.2870 - accuracy: 0.5075 - val_loss: 1.3670 - val_accuracy: 0.4800
Epoch 3/3
7/7 [==============================] - 17s 3s/step - loss: 1.1513 - accuracy: 0.6225 - val_loss: 1.2635 - val_accuracy: 0.5200


In [123]:
preds = model.predict(test_X)
y_pred = np.argmax(preds, axis = 1)

(100, 5)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 3, 4, 0, 1, 0, 0, 0, 2, 0])

In [131]:
import sklearn
precision, recall, f1, _ = sklearn.metrics.precision_recall_fscore_support(test_Y, y_pred, average = 'micro')
print(f'Precision: {precision}\nRecall: {recall}\nF1: {f1}')

Precision: 0.52
Recall: 0.52
F1: 0.52
